#### Backward generation(BWD)

1. generate data after integral

In [1]:
import time

import numpy as np

from backward import generate_bwd
import ray

### Testing ...

In [ ]:
#!wc -l ./data/bwd_sample5000.txt

In [ ]:
from multiprocessing.pool import ThreadPool
import ray
# ray.shutdown()
# ray.init()

_FINISH = False

sequences_per_process = 100
process_runs = 8
round_num = 5

print('generating {} samples in total'.format(
    sequences_per_process * process_runs * round_num)
)

start0 = time.time()
for i in range(round_num):
#     ray.shutdown()
    time.sleep(10)
#     ray.init()
    from multiprocessing.pool import ThreadPool
    _FINISH = False
    start = time.time()
    with ThreadPool(processes=14) as p:
        out = []
        r = p.map_async(generate_bwd, [sequences_per_process]*process_runs, callback=out.append)
        r.wait()
        time.sleep(10)
        print('round {} done'.format(i))
        _FINISH = True
        p.terminate()
        p.join()
#     ray.shutdown()    
    filename = "./data_generation/bwd" + str(sequences_per_process * process_runs) + "_" + str(i+101) + ".txt"
    file = open(filename, "w+")
    for j in range(len(out[0])):
        sequence =out[0][j]
        file.writelines(sequence)
    file.close()
    end = time.time()
    print("Finished: {} round in {} seconds".format(i, end-start))
ray.shutdown()
length = time.time()-start0
print("Finished: in {} hours".format(length/3600))  

generating 4000 samples in total
process finished
process finished
process finished
process finished
process finished
process finished
process finished


<pre>
processes=14
sequences_per_process   process_runs         hours for 1 million 
- 16                       4                 94.28
- 4                        16                54.06
- 128                      4                 80.07
- 32                       16                58.18
- 64                       8                 54.56
</pre>